<a href="https://colab.research.google.com/github/kiera-dev/pokedb/blob/main/PokeSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install duckdb requests pokebase

In [4]:
import duckdb
import json
import pandas as pd
import pokebase as pb

from dataclasses import asdict, dataclass, field
from typing import List

# POKEDEX_LIMIT = 151
POKEDEX_LIMIT = 30

@dataclass
class Pokemon:
    """Class representing a Pokedex entry for a given Pokemon."""
    id: int
    name: str
    height: int
    weight: int
    abilities: List[str] = field(default_factory=list)
    moves: List[str] = field(default_factory=list)
    stats: List[dict] = field(default_factory=list)
    types: List[str] = field(default_factory=list)

    def to_dict(self):
      return asdict(self)

    def to_json(self):
      return json.dumps(asdict(self))

# Populate the Pokedex in DuckDB
pokedex = []
for pokedex_num in range(1, POKEDEX_LIMIT):
  pokemon_metadata = pb.pokemon(pokedex_num)
  abilities = [x.ability.name for x in pokemon_metadata.abilities]
  moves = [x.move.name for x in pokemon_metadata.moves]
  types = [x.type.name for x in pokemon_metadata.types]
  stats = []
  for stat in pokemon_metadata.stats:
    stats.append({stat.stat.name: stat.base_stat})

  pokemon = Pokemon(
      id=pokemon_metadata.id,
      name=pokemon_metadata.name,
      height=pokemon_metadata.height,
      weight=pokemon_metadata.weight,
      abilities=abilities,
      moves=moves,
      stats=stats,
      types=types,
  )
  pokedex.append(pokemon.to_dict())

pokedex_df = pd.DataFrame(pokedex)

In [18]:
duckdb.sql("SELECT * FROM pokedex_df WHERE 'poison' = ANY(types)")

┌───────┬───────────┬────────┬────────┬───┬──────────────────────┬──────────────────────┬─────────────────┐
│  id   │   name    │ height │ weight │ … │        moves         │        stats         │      types      │
│ int64 │  varchar  │ int64  │ int64  │   │      varchar[]       │ map(varchar, integ…  │    varchar[]    │
├───────┼───────────┼────────┼────────┼───┼──────────────────────┼──────────────────────┼─────────────────┤
│     1 │ bulbasaur │      7 │     69 │ … │ [razor-wind, sword…  │ [{hp=45}, {attack=…  │ [grass, poison] │
│     2 │ ivysaur   │     10 │    130 │ … │ [swords-dance, cut…  │ [{hp=60}, {attack=…  │ [grass, poison] │
│     3 │ venusaur  │     20 │   1000 │ … │ [swords-dance, cut…  │ [{hp=80}, {attack=…  │ [grass, poison] │
│    13 │ weedle    │      3 │     32 │ … │ [poison-sting, str…  │ [{hp=40}, {attack=…  │ [bug, poison]   │
│    14 │ kakuna    │      6 │    100 │ … │ [string-shot, hard…  │ [{hp=45}, {attack=…  │ [bug, poison]   │
│    15 │ beedrill  │     10